# Run functions from QCrBoxTools from QCrBox
QCrBoxTools is a library that is installed in any QCrBox container. At the moment it contains three types of functionality. Firstly, functionality to modify, trim, merge or convert the cif entries if crystallographic information framework (cif) files. Secondly, implementations to automatically run some programs we have added to the QCrBox (which are the "Robot" classes.) Finally, functions which are needed in a pipeline workflow of QCrBox. The latter functionality is exposed via the QCrBoxTools container.

## Prerequisites
This notebook uses the `qcrbox_wrapper` package. Make sure you have run

```bash
pip install -e ./qcrbox_wrapper[env-file]
```

from the QCrBox base directory in whatever python environment you are using to run this notebook.

## Import and setting up folders / paths

In [ ]:
import shutil

from qcrbox_wrapper import QCrBoxWrapper, QCrBoxPathHelper

We create an example folder to try out this functionality.

Using the dotenv package makes things more convenient as we can read the environment variables from the .env.dev file in the QCrBox directory. If you want to run with python core packages only, use the `__init__` method instead by defining the path to the shared directory explicitely in `path_to_shared_dir` and replacing the next four lines with:

```python
pathhelper = QCrBoxPathHelper(
    path_to_shared_dir,
    'examples_qcrboxtools'
)
```

In [ ]:
pathhelper = QCrBoxPathHelper.from_dotenv(
    '.env.dev',
    'examples_qcrboxtools'
)

path_local = pathhelper.local_path
path_qcrbox = pathhelper.qcrbox_path

## Connecting to QCrBox
We can connect to the QCrBox Inventory via python after we have started everything with qcb up. Sometimes the server takes a while so you might need to retry if it initially refuses connection. This should not take more than 30 seconds after your console output says that everything has started.

In [ ]:
qcrbox = QCrBoxWrapper('127.0.0.1', 11000)

In [ ]:
qcrboxtools = qcrbox.application_dict['QCrBoxTools']

In [ ]:
help(qcrboxtools)

## Converting a given cif to the unified keywords used in QCrBoxTools

QCrBox works with a subsect of cif entries, which are the base cif entries in the current cif dictionaries (as of February 2024). Additionally uncertainties are split into their own entries. Within the individual containers, the unified cif is then converted to whatever input the individual programs need. To convert any cif file into this format we can use the `to_unified_cif` method of QCrBoxTools. If you prefer to use an outside library instead up spinning up a container, the QCrBoxtools python library also offers this functionality in the `cif_file_to_unified` function of the `qcrboxtools.cif.cif2cif` module.

Let us copy an example file and do the transformation. Notice that calculations in QCrBox are started asynchronously. We can track the status of the calculation using the QCrBoxCalculation object returned by the command call. The QCrBoxCalculation also has a `wait_while_running` method that allows us to stop our script until the command has been completed.

In [ ]:
# create a new folder for this example
folder_unify = path_local / 'run_unify_cif'
folder_unify.mkdir(exist_ok=True)

# copy file
shutil.copy('./input_files/non_unified_kws.cif', folder_unify / 'non_unified_kws.cif')

# start command
calc = qcrboxtools.to_unified_cif(
    input_cif_path=path_qcrbox / 'run_unify_cif' / 'non_unified_kws.cif',
    output_cif_path=path_qcrbox / 'run_unify_cif' / 'output.cif',
    custom_category_list='iucr olex shelx'
)

# wait for command to finish
print(calc.status)
calc.wait_while_running(0.2)
print(calc.status)

Feel free to check the `output.cif`. You can now used this file to start any other functionality within QCrBox.

## Making atoms anisotropic

There is a helper function to make selected atoms anisotropic. Here we can select a dataset, by name or index. The string given in dataset will always be tried as a name of a datablock (in case one of your datablocks is named as a number). If there is no data block of that name, the function will then try to interpret the `cif_dataset` as an index starting from 0 for the first dataset.

In [ ]:
# create a new folder for this example
folder_iso2aniso = path_local / 'run_iso2aniso'
folder_iso2aniso.mkdir(exist_ok=True)

# copy files
shutil.copy('./input_files/iso2aniso.cif', folder_iso2aniso / 'iso2aniso.cif')

# run command
calc2 = qcrboxtools.iso2aniso(
    input_cif_path=path_qcrbox / 'run_iso2aniso' / 'iso2aniso.cif',
    output_cif_path=path_qcrbox / 'run_iso2aniso' / 'output.cif',
    select_names='None',
    select_elements='H',
    select_regexes='None'
)

In [ ]:
print(calc2.status)
calc2.wait_while_running(0.2)
print(calc2.status)

# Functions for use within a data processing pipeline
## Run a convergence check between two cif files


Parameters: Setting a value to `None` will deactivate that test
 - `max_abs_position` : Maximum difference in atomic positions in Angstrom
 - `max_position_su` : Maximum position difference / position su
 - `max abs uij`: Maximum difference in anisotropic displacement parameters in Ang**2
 - `max uij su`: Maximum Uij difference / Uij su

 Output will be written to `output_json` in json format

In [ ]:
# create a new folder for this example
folder_convergence = path_local / 'run_convergence'
folder_convergence.mkdir(exist_ok=True)

# copy files
shutil.copy('./input_files/difference_test1.cif', folder_convergence / 'difference_test1.cif')
shutil.copy('./input_files/difference_test2.cif', folder_convergence / 'difference_test2.cif')

# start calculation
calc3 = qcrboxtools.check_structure_convergence(
    cif1_path=path_qcrbox / 'run_convergence' / 'difference_test1.cif',
    cif2_path=path_qcrbox / 'run_convergence' / 'difference_test2.cif',
    max_abs_position='0.001',
    max_position_su='None',
    max_abs_uij='0.005',
    max_uij_su='1.0',
    output_json=path_qcrbox / 'run_convergence' / 'output.json'
)

# wait for calculation to finish (should be really quick here)
calc3.wait_while_running(0.2)
print(calc3.status)

## Replace structure from cif

replaces structure in input_cif_path with atomic structure (atom_site_table) from structure structure_cif_path. Will be extended in the future.

In [ ]:
calc4 = qcrboxtools.replace_structure_from_cif(
    input_cif_path='/mnt/qcrbox/shared_files/replace_test/test.cif',
    structure_cif_path='/mnt/qcrbox/shared_files/replace_test/80K_P_out.cif'
)

In [ ]:
print(calc4.status)
calc4.wait_while_running(0.2)
print(calc4.status)